# Note before running

API KEY will not be pushed to the repo, you should make one for yourself at https://beta.elevenlabs.io/

In [1]:
import subprocess
import csv
import json
import os
import io
import openai
import boto3

import pandas as pd
import numpy as np
from pytube import YouTube
from elevenlabslib import ElevenLabsUser

from experiments.asr.asr import whisper_asr
from experiments.tts.tts import elevenlabs_tts
from experiments.summarization.openai.summarize import summarize_text, num_tokens_from_text

d:\Egyetem\ELTE\3.felev\AdvancedSoftwareTechnology\podcast-summarization-app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kissa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def list_available_voices(API_KEY):
    user = ElevenLabsUser(API_KEY)
    for voice in user.get_available_voices():
        print(voice.get_name())

In [3]:
def download_video(video_url, destination):
    yt = YouTube(video_url, use_oauth=True, allow_oauth_cache=True)
    print(f'Downloading video: {yt.title} from author {yt.author}')
    video = yt.streams.filter(only_audio=True).first()
    out_file = video.download(output_path=destination)
    base, ext = os.path.splitext(out_file)
    new_target_file_name = base + '.wav'
    subprocess.run(['ffmpeg', '-i', out_file, '-ar', '16000', '-ac', '1', '-b:a', '96K', '-acodec', 'pcm_s16le', new_target_file_name])

    return new_target_file_name

In [4]:
def load_files_and_timestamps(target_dir, target_file_name, output_directory):
    '''
    Reading back the text files. This should be done later asynchronously/with multithreading
    Also saves the transcription df to @param output_directory
    '''
    target_file_name_base, ext = os.path.splitext(target_file_name)
    dfs = []

    #This trick is needed to sort the filenames by index instead of alphabetically
    correct_files = []
    for file_name in os.listdir(target_dir):
        if target_file_name_base in file_name and file_name.endswith('csv'):
            correct_files.append(file_name)
    
    base_file_name = correct_files[0][:correct_files[0].rfind('_')]
    for file_idx in range(len(correct_files)):
        file_name = base_file_name+f'_{file_idx}'+'.csv'
        print(file_name)
        dfs.append(pd.read_csv(os.path.join(target_dir, file_name), delimiter=';', names=['start', 'end', 'text'], encoding='ISO-8859-1', quoting=csv.QUOTE_NONE,))

    df = pd.concat(dfs).reset_index(drop=True)

    final_lines = ' '.join(df['text'])

    df['text_token_counts'] =  df['text'].map(num_tokens_from_text)
    df['token_sum'] = np.cumsum(df['text_token_counts'])

    if not os.path.isdir(output_directory):
        os.mkdir(output_directory)
    df.to_csv(os.path.join(output_directory, target_file_name_base + '_transcription.csv'), index=False, sep=';')

    token_sums = [0] + list(df['token_sum'])
    timestamp_values = list(df['end'])
    timestamp_values.insert(0, df['start'].iloc[0])
    timestamps_dict = dict(zip(token_sums, timestamp_values))

    return final_lines, timestamps_dict

In [5]:
#input parameters

video_url = 'https://www.youtube.com/watch?v=LqjVMy2qhRY'
video_download_folder = 'downloaded_videos_test'
output_directory = 'autogpt_karoly'

ELEVENLABS_API_KEY = ''
AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
openai.api_key = ''

In [6]:
#Download video

target_file_name = download_video(video_url, video_download_folder)
target_file_name = os.path.basename(target_file_name)
target_file_name_base, ext = os.path.splitext(target_file_name)
podcast_sound_path = os.path.join(video_download_folder, target_file_name)

In [7]:
#Speech to text

whisper_asr(podcast_sound_path)

85.20606699999917
77.80173500000092
11.16660380000394


In [8]:
#Loading transcriptions, saving it

text, timestamps_dict = load_files_and_timestamps(video_download_folder, target_file_name, output_directory)

AutoGPT This Is ChatGPT Supercharged!.wav_0.csv
AutoGPT This Is ChatGPT Supercharged!.wav_1.csv
AutoGPT This Is ChatGPT Supercharged!.wav_2.csv


In [9]:
#Text summary, saving the generated files to json

input_text, chunks, chunk_start_timestamps = summarize_text(text, timestamps_dict)

with open(os.path.join(output_directory, target_file_name_base + '_summarized_text.json'), 'w') as f:
    json.dump(input_text, f, indent=2)
with open(os.path.join(output_directory, target_file_name_base + '_chunks.json'), 'w') as f:
    json.dump(chunks, f, indent=2)
with open(os.path.join(output_directory, target_file_name_base + '_chunk_start_timestamps.json'), 'w') as f:
    json.dump(chunk_start_timestamps, f, indent=2)


Text is 1308 tokens long.
Final chunks token counts:  [1308]
Split the text into 1 chunks.
chunk_start_timestamps ['00:00:00.000']
Cost of summary was:  0.002996


In [10]:
#Check if ElevenLabs can be used

summary_length = sum([len(chunk) for chunk in chunks])
print('Summary length:', summary_length)

user = ElevenLabsUser(ELEVENLABS_API_KEY)
remaining_characters = user.get_character_limit() - user.get_current_character_count()
print('Remaining ElevenLabs characters:', remaining_characters)

if summary_length > remaining_characters:
    raise ValueError('Not enough characters for TTS. Provide an ElevenLabs API token with enough remaining characters.')

Summary length: 652
Remaining ElevenLabs characters: 6554


In [11]:
#TTS with elevenlabs

elevenlabs_tts(chunks, ELEVENLABS_API_KEY, 'Adam', os.path.join(output_directory, target_file_name_base + '_read_summary.mp3'))#Male voice

In [12]:
#upload to s3 bucket

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)
s3 = session.resource('s3')
bucket_name = 'breviocast-prod'

In [15]:
def list_files_in_bucket(bucket_name, s3):
    my_bucket = s3.Bucket(bucket_name)
    for my_bucket_object in my_bucket.objects.all():
        print(my_bucket_object.key)

def upload_directory_to_bucket(path, bucket_name, s3):
    for file_name in os.listdir(path):
        s3.meta.client.upload_file(os.path.join(path,file_name), bucket_name, f'podcasts/{path}/{file_name}')

upload_directory_to_bucket(output_directory, bucket_name, s3)
list_files_in_bucket(bucket_name, s3)

podcasts/uranium_mining_outputs/How Uranium Mining Works  STUFF YOU SHOULD KNOW_chunk_start_timestamps.json
podcasts/uranium_mining_outputs/How Uranium Mining Works  STUFF YOU SHOULD KNOW_chunks.json
podcasts/uranium_mining_outputs/How Uranium Mining Works  STUFF YOU SHOULD KNOW_read_summary.mp3
podcasts/uranium_mining_outputs/How Uranium Mining Works  STUFF YOU SHOULD KNOW_summarized_text.json
podcasts/uranium_mining_outputs/How Uranium Mining Works  STUFF YOU SHOULD KNOW_transcription.csv
test.txt
uranium_mining_outputs\How Uranium Mining Works  STUFF YOU SHOULD KNOW_read_summary.mp3
uranium_mining_outputs\How Uranium Mining Works  STUFF YOU SHOULD KNOW_transcription.csv
uranium_mining_outputs\How Uranium Mining Works  STUFF YOU SHOULD KNOWchunk_start_timestamps.json
uranium_mining_outputs\How Uranium Mining Works  STUFF YOU SHOULD KNOWchunks.json
uranium_mining_outputs\How Uranium Mining Works  STUFF YOU SHOULD KNOWsummarized_text.json
